#Data mining the hansard

Estimating contribution by party

## Getting debate text
- mine hansard from theyworkforyou scrape for June 2015 onwards using lxml

In [24]:
import sys
sys.path.append('/home/fin/Documents/datamining_hansard/dh/lib/python3.4/site-packages/')

In [27]:
import lxml
import requests

In [29]:
page = requests.get("http://www.theyworkforyou.com/pwdata/scrapedxml/debates/")
tree = lxml.html.fromstring(page.text)

debates = tree.xpath('//a/text()')
debates2015 = [x for x in debates if x.startswith('debates2015')]
new_parliament = [x for x in debates2015 if int(x.split('-')[1]) > 4]


In [49]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
    return local_filename

for xml in new_parliament:
    download_file("http://www.theyworkforyou.com/pwdata/scrapedxml/debates/"+xml)

## Parsing xml of debates
- XML acquired now to start parsing it using lxml

In [58]:
def get_speaker_and_text(xml):
    """
    For a given xml file return a list of tuples
    of speeches and their speakers
    """
    with open(xml, 'rb') as fh:
        parsed_xml = fh.read()
    
    data = []
    root = lxml.etree.fromstring(parsed_xml)
    speeches = root.xpath('speech') 
    for speech in speeches:
        #if speech.get('nospeaker') is not None:
        speech_list = [x.text for x in speech.getchildren() if x.text is not None]
        
        speech_list = " ".join(speech_list)
        
        speaker =  speech.get('speakername')
        data.append((speaker, speech_list))
    
    return data


## Get MP name and party affiliation

- Get a dict of MP names: party affiliation

In [246]:
import csv
with open('mp_data.csv',  encoding = "ISO-8859-1") as fh:
    dat = [x for x in csv.reader(fh)]

party_dict = {}
for mp in dat[1:]:
    name = mp[1] + ' ' + mp[2]
    party = mp[3]
    party_dict.update({name: party})
    
#add the speakers
party_dict.update({'Mr Speaker': 'Mr Speaker'})
party_dict.update({'Mr Deputy Speaker': 'Mr Speaker'})
party_dict.update({'Mr Speaker-Elect': 'Mr Speaker'})

#add Stephen Philip Rotheram for some reaosn
party_dict.update({'Stephen Philip Rotheram': 'Labour'})


## Parsin' time

- Time to parse the shit out of this parliament

In [244]:
import glob

named_text = []
for xml in glob.glob("*.xml"):
    named_text = named_text + get_speaker_and_text(xml)

In [245]:
# get complete text for each party
party_talk = {}
for party in set(party_dict.values()):
    party_talk.update({party:''})
    
for speech in named_text:
    if speech[0] in party_dict.keys():
        party = party_dict[speech[0]]
        party_talk[party] = party_talk[party] + speech[1] 
    else:
        print(speech[0])

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
My Lords and Members of the House of Commons
Members of the House of Commons
My Lords and Members of the House of Commons
My Lords and Members of the House of Commons
None
None
None
Hon. Members
None
None
None
None
None
None
Hon. Members
Hon. Members
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Hon. Members
None
None
None
None
None
None
None
Hon. Members
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
The Chairman
The Chairman
The Chairman
The Chairman
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
N

In [262]:
print(len(set(party_talk['Scottish National Party']))/56, len(party_talk['Scottish National Party'])/56)

1.6785714285714286 24963.285714285714


In [263]:
print(len(set(party_talk['Labour']))/232, len(party_talk['Labour'])/232)

0.4353448275862069 14524.112068965518


In [266]:
print(len(set(party_talk['Conservative']))/330, print(len(party_talk['Conservative'])/330))

18988.484848484848
0.3212121212121212 None
